In [1]:
import tensorflow as tf
from model.fcn import fcnRegressor
from config_folder_guard import config_folder_guard
from gen_batches import gen_batches
from logger import my_logger as logger
import pandas as pd
from PIL import Image

In [2]:
def read_csv_file():


    DATASET_MEDIUM_DIR = 'dataset_medium.csv'
    Image.MAX_IMAGE_PIXELS = None

    imgs_dirs = []
    dataset_read_result = pd.read_csv(DATASET_MEDIUM_DIR)
    i = 0
    for each_img_dir, \
        each_landmarks_dir, \
        each_target_image, \
        each_target_landmarks, \
        each_status in zip(dataset_read_result['Source image'],
                           dataset_read_result['Source landmarks'],
                           dataset_read_result['Target image'],
                           dataset_read_result['Target landmarks'],
                           dataset_read_result['status']):
        each_img_dir = 'images/' + each_img_dir
        each_landmarks_dir = 'landmarks/' + each_landmarks_dir
        each_target_image = 'images/' + each_target_image
        each_target_landmarks = 'landmarks/' + each_target_landmarks

        dataset_read_result.set_value(index=i, col='Source image', value=each_img_dir)
        dataset_read_result.set_value(index=i, col='Source landmarks', value=each_landmarks_dir)
        dataset_read_result.set_value(index=i, col='Target image', value=each_target_image)
        dataset_read_result.set_value(index=i, col='Target landmarks', value=each_target_landmarks)

        imgs_dirs.append(each_img_dir)
        i = i + 1

    print(dataset_read_result['Source image'][1])
    print(imgs_dirs[1])
    return dataset_read_result


dataset_read_result = read_csv_file()

images/COAD_01/scale-25pc/S3.jpg
images/COAD_01/scale-25pc/S3.jpg


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [7]:
def gen_batches(dataset_read_result):
    source_image_array = dataset_read_result['Source image']
    target_image_array = dataset_read_result['Target image']

    # 如果参考图像数量和待配准图像数量不同，那么意味着出错了
    assert len(source_image_array) == len(target_image_array)

    source_image_array = tf.cast(source_image_array, tf.string)
    target_image_array = tf.cast(target_image_array, tf.string)

    input_queue = tf.train.slice_input_producer([source_image_array, target_image_array])
    source_image_array = tf.read_file(input_queue[0])
    target_image_array = tf.read_file(input_queue[1])
    source_image_array = tf.image.decode_jpeg(source_image_array, channels=3)
    target_image_array = tf.image.decode_jpeg(target_image_array, channels=3)

    # resize
    source_image_array = tf.image.resize_images(source_image_array, [128, 128], method=tf.image.ResizeMethod.BICUBIC)
    target_image_array = tf.image.resize_images(target_image_array, [128, 128], method=tf.image.ResizeMethod.BICUBIC)

    source_image_array = tf.cast(source_image_array, tf.float32)
    target_image_array = tf.cast(target_image_array, tf.float32)

    source_image_array = tf.image.per_image_standardization(source_image_array)
    target_image_array = tf.image.per_image_standardization(target_image_array)

    # 标准化数据
    source_image_array, target_image_array = tf.train.batch([source_image_array, target_image_array],
                                                            batch_size=10,
                                                            num_threads=32,  # 线程
                                                            capacity=256)
    return source_image_array, target_image_array


source_image_array, target_image_array = gen_batches(dataset_read_result)


In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
source_image_array, target_image_array = sess.run([source_image_array, target_image_array])

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.ResourceExhaustedError'>, OOM when allocating tensor with shape[12747,11956,3] and type uint8 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cuda_host_bfc
	 [[{{node DecodeJpeg_1}} = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

